# Assignment_2

## 问题
- Data: W3C HTML5 中文兴趣小组一个月的邮件存档, 格式mbox
- 试定位邮件中的签名档, 并尽可能提取多的字段


## 思考
- 是不是应该考虑先来一波暴力提取
    - 确实思路是层层递进, 先粗略来一波, 慢慢精确到想得到的信息. 记住, MVP!!!
- 观察数据时看到很多乱码, 涉不涉及转码?
    - 其实就是中文, 还有图片的码.
- 每封邮件的起止位置标识是什么?
    - 答: From ....@......
    - 其实还有boundary, 但是发现用boundary不如用From精确.
- 如何尽可能多的提取信息?
    - 目前来看, 就是不断的根据数据添加规则:if, elif...
- 新思路: 用分词提取所有姓名, 然后根据每一条邮件内容提取签名档
    - 这么做可行, 虽然很慢. 但是由于每封邮件里的姓名不统一, 即From 的名字跟签名档的名字不同, 针对每个名字去添加列表太慢, 
    - 反而这样暴力提取所有, 在根据特别的名字进行补充, 这样更节省操作的复杂度.
- 因为提取的姓名格式有几个形式不同, 比如 中英混杂, 名字中间带括号.
    - 所以考虑先生成一份处理过的name_list, 然后, 编译这份name_list到正则表达式.
    - 但是这么做应该会导致程序比较慢, 尤其是当name_list长度特别大时. 
    - 可以通过观察name_list, 发现里面很多条目是重复的, 因为邮件互相往来嘛... 本着'最小'的理念, 先试试!
- 另外一个想法:
    - 对全文进行分词, 提取人名, 加之前通过Flanker提取的人名得到一份名单.
    - 在week2中对正文进行的分词, 这样不可取, 因为虽然是中文交流. 但签名档用的是英文. 用NLTK去分词小效果并不会很好.
- 签名档没有固定格式, 感觉找不到一个统一的规则去提取签名档...心塞!
    - 难道要写一个巨能容错的正则(规则复杂, 冗长)去匹配, 然后在筛选?


## 过程
- 根据mbox格式分开每一封邮件
- 利用flanker print 邮件内容, 观察是否有一般的提取模式
    - 期间遇到了编码问题, 有NoneType, 以及str, 还有Unicode.
- 发现无法利用内容提取, 
- 如何提取发件人姓名?
    - 利用flanker在header里面找到'From', 可以提取发件人姓名, 邮箱.
    - 由于有的发件人姓名与签名单姓名不符, 采用了手动添加.
- 生成一份name list, 然后用Regex来一波暴力提取, 即将所以姓名后边的全取出来
- 然后根据长度, 先过滤掉过长的.
- 通过观察剩下的string, 就可以有针对性的进行筛选
    - html里的就用beautifulSoup
    - 不是签名档的直接删除
- 最后得出一份比较粗的签名档, 由于这份数据的签名档太少, 且格式没用共性, 没用继续进一步的提取.

## References
- [mbox-wikipedia](https://en.wikipedia.org/wiki/Mbox)


In [1]:
# -*- coding: utf-8 -*-
import os
import re
from nltk.tag import StanfordNERTagger
import flanker

from bs4 import BeautifulSoup
from flanker import mime

In [2]:
# Add models of NLTK  
os.environ["CLASSPATH"] = "/Users/xpgeng/Library/stanford-ner-2015-12-09"  
os.environ["STANFORD_MODELS"] = "/Users/xpgeng/Library/stanford-ner-2015-12-09/models"

In [3]:
# Add Tagger
st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz')

In [10]:
def prepare_data(filename='2013-11.mbx'):
    '''
    Return the email list
    '''
    with open(filename, 'r') as f:
        data = f.read()
    f.close()
    data_list = filter(None, re.split(r'From\s([\w+.?]+@(\w+\.)+(\w+))', data))  #  
    # Here I have to add twice for-loop, I haven't analysed the reason
    for data in data_list:
        if len(str(data)) < 500:
            data_list.remove(data)
    for data in data_list:
        if len(str(data)) < 500:
            data_list.remove(data)
    return data_list 

In [5]:
data_list = prepare_data('2013-11.mbx')

In [6]:
def create_name_list(data_list):
    '''
    Using regex to extract name,  add and remove some special names 
    '''
    name_list = []
    p = re.compile(ur'\"?([\w\s\(\)]+|[\x80-\xff]+)\"?\s<')
    for message_string in data_list:
        msg = mime.from_string(message_string)
        for item in msg.headers.items():
            if item[0] == 'From':
                name = p.search(item[1].encode('utf-8')).group(1)
                name_list.append(name)
    name_list = list(set(name_list))
    name_list += ['Cindy', 'Kenny', 'Chen Yijun', 'Chunming', '-ambrose']
    name_list.remove('com')
    name_list.remove(' Chunming')
    name_list.remove(' Bobby Tung')
    name_list.remove('Hawkeyes Wind')
    return name_list

In [7]:
def extract_signature(message_string, name_list):
    '''
    Parameters:
        message_string: all the information of an email
        name_list: all the senders 
    Returns:
        signature: str
    '''
    signature_list = []
    for name in name_list:
        p_name = re.compile(r'^%s.+'%name, re.MULTILINE | re.DOTALL)
        msg = mime.from_string(message_string)
        for part in msg.parts:
            if not isinstance(part.body, (type(None), str)):
                if p_name.findall(part.body.encode('utf-8')):
                    signature_list += p_name.findall(part.body.encode('utf-8'))
    signature = None
    for item in signature_list:
        if len(item) < 300: 
            signature = item # 已经知道小于300的就一个
    if not signature:
        return None
    elif 'Hawkeyes Wind' in signature or 'Zhiqiang' in signature: # 只能不断添加规则...
        return None
    elif '<' in signature:
        soup = BeautifulSoup(item, 'html.parser')
        signature = soup.get_text()
        return signature
    else:
        return signature
    

In [8]:
name_list = create_name_list(data_list)

In [1]:
signature_list = []
for data in data_list:
    signature_list.append(extract_signature(data, name_list))
signature_list = filter(None, signature_list)
signature_list = set(signature_list)
for item in signature_list:
    print item

NameError: name 'data_list' is not defined